# Using tensorflow like numpy

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


## Constants

In [2]:
matrix1 = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("matrix1: \n", matrix1)
# Just like an ndarray , a tf.Tensor has a shape and a data type
print("matrix1.shape: ", matrix1.shape)
print("matrix1.dtype: ", matrix1.dtype)
print("matrix1.numpy(): \n",matrix1.numpy())

matrix1: 
 tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
matrix1.shape:  (2, 3)
matrix1.dtype:  <dtype: 'float32'>
matrix1.numpy(): 
 [[1. 2. 3.]
 [4. 5. 6.]]


#### Indexing 

works much like in NumPy

In [3]:
print("matrix1[:, 1:]: ", matrix1[:, 1:])
print("matrix1[..., 1, tf.newaxis] :", matrix1[..., 1, tf.newaxis])
print("matrix1[..., 1]: ", matrix1[..., 1])

matrix1[:, 1:]:  tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
matrix1[..., 1, tf.newaxis] : tf.Tensor(
[[2.]
 [5.]], shape=(2, 1), dtype=float32)
matrix1[..., 1]:  tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [4]:
t = tf.constant([[1, 2], [3, 4]])
print("t: " , t)
print("t+10: ", t+10)
print("t*100: ", t*100)
print("tf.square(t): ", tf.square(t))

# from python 3.5 onwards @ is used for matrix multiplications
print("t@tf.transpose(t): ", t@tf.transpose(t))

t:  tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
t+10:  tf.Tensor(
[[11 12]
 [13 14]], shape=(2, 2), dtype=int32)
t*100:  tf.Tensor(
[[100 200]
 [300 400]], shape=(2, 2), dtype=int32)
tf.square(t):  tf.Tensor(
[[ 1  4]
 [ 9 16]], shape=(2, 2), dtype=int32)
t@tf.transpose(t):  tf.Tensor(
[[ 5 11]
 [11 25]], shape=(2, 2), dtype=int32)


can use other operations like tf.add() , tf.multiply() ,
tf.square() , tf.exp() , tf.sqrt()
tf.reshape() , tf.squeeze() , tf.tile()
tf.reduce_mean() , tf.reduce_sum() , tf.reduce_max() ,tf.math.log()

**Notice that NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit. This is because 32-bit precision is generally more than enough for neural networks, plus it runs faster and uses less RAM. So when you create a tensor from a NumPy array, make sure to set dtype=tf.float32**

### Type conversions 

Ican significantly hurt performance, and they can easily go unnoticed when they are done automatically. 
To avoid this, TensorFlow does not perform any type conversions automatically: it just raises an exception if you try to execute an operation on tensors with incompatible types. 

For example, you cannot add a float tensor and an integer tensor, and you cannot even add a 32-bit float and a 64-bit float:

In [6]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute Add as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Add] name: add/

In [7]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute Add as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:Add] name: add/

In [8]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: id=39, shape=(), dtype=float32, numpy=42.0>

## Variables

The weights in a neural network need to be tweaked by backpropagation, and other parameters may also need to change over time (e.g., a momentum optimizer keeps track of past gradients). What we need is a tf.Variable :

In [9]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [16]:
print("v: ", v)
print("v.dtype: ", v.dtype)
print("v.shape: ", v.shape)
print("v.numpy(): ", v.numpy())
print("v.name: ", v.name)

v:  <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
v.dtype:  <dtype: 'float32'>
v.shape:  (2, 3)
v.numpy():  [[1. 2. 3.]
 [4. 5. 6.]]
v.name:  Variable:0


The tf.Variable is like tf.Constant, but it can also be modified in place using the assign() 
method (or assign_add() or assign_sub() which increment or decrement the variable by the given value). 
You can also modify individual cells (or slices), using the cell’s (or slice’s) assign() 
method (direct item assignment will not work), or using the scatter_update() or scatter_nd_update() methods:

In [22]:
# changes 'v' in place
v.assign(2 * v)    # => [[2., 4., 6.], [8., 10., 12.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4.,  8., 12.],
       [16., 20., 24.]], dtype=float32)>

In [23]:
v[0, 1].assign(42)  # => [[2., 42., 6.], [8., 10., 12.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42., 12.],
       [16., 20., 24.]], dtype=float32)>

In [24]:
v[:, 2].assign([0., 1.]) # => [[2., 42., 0.], [8., 10., 1.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 4., 42.,  0.],
       [16., 20.,  1.]], dtype=float32)>

In [25]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])
# => [[100., 42., 0.], [8., 10., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [ 16.,  20., 200.]], dtype=float32)>

## Other Data Structures

• Sparse tensors ( tf.SparseTensor ) efficiently represent tensors containing mostly
0s. The tf.sparse package contains operations for sparse tensors.

• Tensor arrays ( tf.TensorArray ) are lists of tensors. They have a fixed size by
default, but can optionally be made dynamic. All tensors they contain must have
the same shape and data type.

• Ragged tensors ( tf.RaggedTensor ) represent static lists of lists of tensors, where
every tensor has the same shape and data type. The tf.ragged package contains
operations for ragged tensors.

• String tensors are regular tensors of type tf.string . These actually represent byte
strings, not Unicode strings, so if you create a string tensor using a Unicode
string (e.g., a regular Python 3 string like "café"` ), then it will get encoded to
UTF-8 automatically (e.g., b"caf\xc3\xa9" ). Alternatively, you can represent
Unicode strings using tensors of type tf.int32 , where each item represents a
Unicode codepoint (e.g., [99, 97, 102, 233] ). The tf.strings package (with
an s ) contains ops for byte strings and Unicode strings (and to convert one into
the other).

• Sets are just represented as regular tensors (or sparse tensors) containing one or
more sets, and you can manipulate them using operations from the tf.sets
package.

• Queues, including First In, First Out (FIFO) queues ( FIFOQueue ), queues that can
prioritize some items ( PriorityQueue ), queues that shuffle their items ( Random
ShuffleQueue ), and queues that can batch items of different shapes by padding
( PaddingFIFOQueue ). These classes are all in the tf.queue package.

### String tensors

It will be really useful when generating a data pipeline and tfrecord generation using tf.data api

In [37]:
a = tf.constant(1234)
print("a.numpy(): ", a.numpy())
s1 = tf.strings.as_string(a)
print("s1: ", s1)

a.numpy():  1234
s1:  tf.Tensor(b'1234', shape=(), dtype=string)


In [38]:
s2 = tf.constant('asdf')
print("s2: ", s2)
print("s2.dtype: ", s2.dtype)

s2:  tf.Tensor(b'asdf', shape=(), dtype=string)
s2.dtype:  <dtype: 'string'>


In [39]:
s3 = tf.constant(['hello', 'TensorFlow', 'world!'])

In [45]:
s4 = tf.strings.join(s3, separator='*')
print(s4.numpy())
print("length of s4: ", tf.strings.length(s4).numpy())

b'hello*TensorFlow*world!'
length of s4:  23


In [49]:
s5 = tf.strings.split(s4, sep="*")
print(s5.numpy())

[b'hello' b'TensorFlow' b'world!']
